In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/05 14:54:13 WARN Utils: Your hostname, sara-latitude-7490 resolves to a loopback address: 127.0.1.1; using 10.0.0.253 instead (on interface wlp2s0)
23/03/05 14:54:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 14:54:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/05 14:54:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.version

'3.3.2'

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz

In [4]:
!ls -lh "fhv_tripdata_2021-06.csv"

-rw-r--r-- 1 sara sara 878M Dec 14 18:00 /home/sara/Downloads/fhv_tripdata_2021-06.csv


In [6]:
# schema = types.StructType([
#     types.StructField('hvfhs_license_num', types.StringType(), True),
#     types.StructField('dispatching_base_num', types.StringType(), True),
#     types.StructField('pickup_datetime', types.TimestampType(), True),
#     types.StructField('dropoff_datetime', types.TimestampType(), True),
#     types.StructField('PULocationID', types.IntegerType(), True),
#     types.StructField('DOLocationID', types.IntegerType(), True),
#     types.StructField('SR_Flag', types.StringType(), True)
# ])

In [12]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema" , "true") \
    .csv('fhv_tripdata_2021-06.csv')

df = df.repartition(12)

df.write.parquet('./fhvhv_partitions/')

23/03/05 15:43:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [22]:
df = spark.read.parquet('./fhvhv_partitions/')
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



**Q3**: How many taxi trips were there on June 15?

In [15]:
from pyspark.sql import functions as F

In [17]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

In [24]:
df.registerTempTable('fhvhv_2021_06')

In [19]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2021_06
WHERE
    to_date(pickup_datetime) = '2021-06-15';
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



**Q4**: Longest trip for each day

In [20]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [23]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-06-25|       240764|
| 2021-06-22|        91979|
| 2021-06-27|        71931|
| 2021-06-26|        65510|
| 2021-06-23|        59281|
+-----------+-------------+



**Q6**: Most frequent pickup location zone

In [27]:
df_zones = spark.read \
    .option("header", "true") \
    .option("inferSchema" , "true") \
    .csv('/home/sara/Downloads/taxi+_zone_lookup.csv')

In [28]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [29]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [30]:
df_zones.registerTempTable('zones')

/home/sara/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [31]:
spark.sql("""
SELECT
    CONCAT(pul.Zone) AS pickup_zone,
    COUNT(1)
FROM 
    fhvhv_2021_06 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+-------------------+--------+
|        pickup_zone|count(1)|
+-------------------+--------+
|Crown Heights North|  231279|
|       East Village|  221244|
|        JFK Airport|  188867|
|     Bushwick South|  187929|
|      East New York|  186780|
+-------------------+--------+

